# Samples: AuthX Pro - Authentication & Authorization

Sets up a more sophisticate authentication (authN) and authorization (authZ) combination for role-based access control (RBAC) to a mock API and its operations.  

This sample, compared to the simpler _AuthX_, introduces use of API Management Product and policy fragments to simplify and consolidate shared logic. When considering scaling, consider this as your starting point.

⚙️ **Supported infrastructures**: All infrastructures

⌚ **Expected *Run All* runtime (excl. infrastructure prerequisite): ~2-3 minutes**

## 🎯 Objectives

1. Understand how API Management supports OAuth 2.0 authentication (authN) with JSON Web Tokens (JWT).
1. Learn how authorization (authZ) can be accomplished based on JWT claims.
1. Configure authN and authZ at various levels in the API Management hierarchy - product, API, and API operations
1. Use external secrets in policies.
1. Experience how API Management policy fragments simplify shared logic.

## 📝 Scenario

This sample combines _authentication (authN)_ and _authorization (authZ)_ into _authX_. This scenario focuses on a Human Resources API that requires privileged role-based access to GET and to POST data. This is simplistic but shows the combination of authN and authZ.

There are two personas at play:

- `HR Administrator` - holds broad rights to the API
- `HR Associate` - has read-only permissions

Both personas are part of an HR_Members group and may access the HR API Management Product. Subsequent access to the APIs and their operations must be granular.

### 💡 Notes

Many organizations require 100% authentication for their APIs. While that is prudent and typically done at the global _All APIs_ level, we refrain from doing so here as to not impact other samples. Instead, we focus on authentication at the API Management API and API operation levels.

## 🧩 Lab Components

While OAuth 2.0 includes an identity provider (IDP), for sake of the sample, we can remove the complexity of including real identities. It is sufficient to use mock JWTs that we can "authenticate" by way of a signing key. This is a valid, albeit not the default method for authentication. 

We do not need real APIs and can rely on mock returns.

Furthermore, secrets would ideally be kept in a secret store such as Azure Key Vault and be accessed via API Management's managed identity. Adding a Key Vault to our architecture is a stretch goal that provides value but is not immediately necessary to showcase the authX sample.

JSON Web Tokens (JWTs) are defined in RFC 7519. Several tools exist to explore JWTs:

🔗 [RFC 7519 - JWT](https://www.rfc-editor.org/rfc/rfc7519) | [jwt.io](https://jwt.io/) | [jwt.ms](https://jwt.ms/)

## ⚙️ Configuration

1. Decide which of the [Infrastructure Architectures](../../README.md#infrastructure-architectures) you wish to use.
    1. If the infrastructure _does not_ yet exist, navigate to the desired [infrastructure](../../infrastructure/) folder and follow its README.md.
    1. If the infrastructure _does_ exist, adjust the `user-defined parameters` in the _Initialize notebook variables_ below. Please ensure that all parameters match your infrastructure.

### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

👉 **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
import time
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['authX-pro', 'jwt', 'policy-fragment']       # ENTER DESCRIPTIVE TAG(S)
api_prefix  = 'authX-pro-'               # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)
supported_infrastructures = [INFRASTRUCTURE.SIMPLE_APIM, INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA]        # ENTER SUPPORTED INFRASTRUCTURES HERE, e.g., [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.AFD_APIM_FE]
utils.validate_infrastructure(deployment, supported_infrastructures)

# Set up the signing key for the JWT policy
jwt_key_name = f'JwtSigningKey{int(time.time())}'
jwt_key_value, jwt_key_value_bytes_b64 = utils.generate_signing_key()
utils.print_val('JWT key value', jwt_key_value)                         # this value is used to create the signed JWT token for requests to APIM
utils.print_val('JWT key value (base64)', jwt_key_value_bytes_b64)      # this value is used in the APIM validate-jwt policy's issuer-signing-key attribute  

# 3) Set up the named values
nvs: List[NamedValue] = [
    NamedValue(jwt_key_name, jwt_key_value_bytes_b64, True),
    NamedValue('HRMemberRoleId', HR_MEMBER_ROLE_ID),
    NamedValue('HRAssociateRoleId', HR_ASSOCIATE_ROLE_ID),
    NamedValue('HRAdministratorRoleId', HR_ADMINISTRATOR_ROLE_ID)
]

# 4) Set up the policy fragments
pf_authx_hr_member_xml = utils.read_policy_xml('./pf-authx-hr-member.xml').format(
    jwt_signing_key = '{{' + jwt_key_name + '}}',
    hr_member_role_id = '{{HRMemberRoleId}}'
)
# TODO: These shared policies may be better applied as part of each infrastructure deployments. 
pf_authz_match_any_xml = utils.read_policy_xml('../../shared/apim-policies/fragments/pf-authz-match-any.xml')
pf_http_response_200_xml = utils.read_policy_xml('../../shared/apim-policies/fragments/pf-http-response-200.xml')

pfs: List[PolicyFragment] = [
    PolicyFragment('AuthX-HR-Member', pf_authx_hr_member_xml, 'Authenticates and authorizes HR members.'),
    PolicyFragment('AuthZ-Match-Any', pf_authz_match_any_xml, 'Authorizes if any of the specified roles match the JWT role claims.'),
    PolicyFragment('Http-Response-200', pf_http_response_200_xml, 'Returns a 200 OK response for the current HTTP method.')
]

# 5) Define the Products (NEW: moved authentication to product level)

# HR Product with authentication policy, including authorization via a required claim check for HR member role
hr_product_xml = utils.read_policy_xml('./hr_product.xml').format(
    jwt_signing_key = '{{' + jwt_key_name + '}}', 
    hr_member_role_id = '{{HRMemberRoleId}}'
)

hr_product_name = 'hr'
products: List[Product] = [
    Product(hr_product_name, 'Human Resources', 'Product for Human Resources APIs providing access to employee data, organizational structure, benefits information, and HR management services. Includes JWT-based authentication for HR members.', 'published', False, False, hr_product_xml)
]

# 6) Define the APIs and their operations and policies

# Employees (HR)
hremployees_api_path = f'/{api_prefix}employees'
hremployees_get = GET_APIOperation('Gets the employees', utils.read_policy_xml('./hr_get.xml'))
hremployees_post = POST_APIOperation('Creates a new employee', utils.read_policy_xml('./hr_post.xml'))
hremployees = API(f'{api_prefix}Employees', 'Employees Pro', hremployees_api_path, 'This is a Human Resources API for employee information', utils.read_policy_xml(DEFAULT_XML_POLICY_PATH), operations = [hremployees_get, hremployees_post], tags = tags, productNames = [hr_product_name])

# Benefits (HR)
hrbenefits_api_path = f'/{api_prefix}benefits'
hrbenefits_get = GET_APIOperation('Gets employee benefits', utils.read_policy_xml('./hr_get.xml'))
hrbenefits_post = POST_APIOperation('Creates employee benefits', utils.read_policy_xml('./hr_post.xml'))
hrbenefits = API(f'{api_prefix}Benefits', 'Benefits Pro', hrbenefits_api_path, 'This is a Human Resources API for employee benefits', utils.read_policy_xml(DEFAULT_XML_POLICY_PATH), operations = [hrbenefits_get, hrbenefits_post], tags = tags, productNames = [hr_product_name])

# APIs Array
apis: List[API] = [hremployees, hrbenefits]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]},
    'policyFragments': {'value': [pf.to_dict() for pf in pfs]},
    'products': {'value': [product.to_dict() for product in products]}
}

# 2) Infrastructure must be in place before samples can be layered on top
if not utils.does_resource_group_exist(rg_name):
    utils.print_error(f'The specified infrastructure resource group and its resources must exist first. Please check that the user-defined parameters above are correctly referencing an existing infrastructure. If it does not yet exist, run the desired infrastructure in the /infra/ folder first.')
    raise SystemExit(1)

# 3) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 4) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests
from apimjwt import JwtPayload, SymmetricJwtToken
from apimtypes import HR_MEMBER_ROLE_ID, HR_ADMINISTRATOR_ROLE_ID, HR_ASSOCIATE_ROLE_ID

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door. If so, assume that APIM is not directly accessible and use the Front Door URL instead.
endpoint_url = apim_gateway_url
utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(deployment, rg_name)

if afd_endpoint_url:
    endpoint_url = afd_endpoint_url
    utils.print_message(f'Using Azure Front Door URL: {afd_endpoint_url}', blank_above = True)
else:
    utils.print_message(f'Using APIM Gateway URL: {apim_gateway_url}', blank_above = True)

# 1) HR Administrator
# Create a JSON Web Token with a payload and sign it with the symmetric key from above.
jwt_payload_hr_admin = JwtPayload(subject = 'user123', name = 'Angie Administrator', roles = [HR_MEMBER_ROLE_ID, HR_ADMINISTRATOR_ROLE_ID])
encoded_jwt_token_hr_admin = SymmetricJwtToken(jwt_key_value, jwt_payload_hr_admin).encode()
print(f'\nJWT token HR Admin: {encoded_jwt_token_hr_admin}')  # this value is used to call the APIs via APIM

# Set up an APIM requests object with the JWT token
reqsApimAdmin = ApimRequests(endpoint_url)
reqsApimAdmin.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_admin}'

# Call APIM
reqsApimAdmin.singleGet(hremployees_api_path, msg = 'Calling GET Employees API via API Management Gateway URL. Expect 200.')
reqsApimAdmin.singlePost(hremployees_api_path, msg = 'Calling POST Employees API via API Management Gateway URL. Expect 200.')
reqsApimAdmin.singleGet(hrbenefits_api_path, msg = 'Calling GET Benefits API via API Management Gateway URL. Expect 200.')
reqsApimAdmin.singlePost(hrbenefits_api_path, msg = 'Calling POST Benefits API via API Management Gateway URL. Expect 200.')

# 2) HR Associate
# Create a JSON Web Token with a payload and sign it with the symmetric key from above.
jwt_payload_hr_associate = JwtPayload(subject = 'user789', name = 'Aaron Associate', roles = [HR_MEMBER_ROLE_ID, HR_ASSOCIATE_ROLE_ID])
encoded_jwt_token_hr_associate = SymmetricJwtToken(jwt_key_value, jwt_payload_hr_associate).encode()
print(f'\nJWT token HR Associate: {encoded_jwt_token_hr_associate}')  # this value is used to call the APIs via APIM

# Set up an APIM requests object with the JWT token
reqsApimAssociate = ApimRequests(endpoint_url)
reqsApimAssociate.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_associate}'

# Call APIM
reqsApimAssociate.singleGet(hremployees_api_path, msg = 'Calling GET Employees API via API Management Gateway URL. Expect 200.')
reqsApimAssociate.singlePost(hremployees_api_path, msg = 'Calling POST Employees API via API Management Gateway URL. Expect 403.')
reqsApimAssociate.singleGet(hrbenefits_api_path, msg = 'Calling GET Benefits API via API Management Gateway URL. Expect 200.')
reqsApimAssociate.singlePost(hrbenefits_api_path, msg = 'Calling POST Benefits API via API Management Gateway URL. Expect 403.')

utils.print_ok('All done!')